# Noms dérivés en *-age* et *-ment* : la concurrence suffixale

---

## &#9888; Attention : ne pas exécuter le code de la cellule ci-dessous

Il s'agit du code qui a permis d'extraire de GLÀFF un **sous-corpus** de 4485 noms en *-age* et *-ment* :

Il s'agit du fichier `glaff_age_ment_light.txt` que vous devez télécharger sur la [page du cours](https://lies-think-c48.craft.me/SnVcy44gczcXLi).

- - -

In [ ]:
import pandas as pd

# 1. Chargement du GLAFF complet
df = pd.read_csv('glaff-1.2.2.txt', sep='|', header=None, on_bad_lines='skip', usecols=[0, 1, 2])
df.columns = ['Graphie', 'Code_GRACE', 'Lemme']

# 2. Filtrage : Noms communs (Nc) finissant par -age ou -ment
filtre = (df['Code_GRACE'].str.startswith('Nc', na=False)) & \
         (df['Graphie'].str.endswith(('age', 'ment'), na=False))

noms_extraits = df[filtre].copy()

# Génération du fichier texte allégé (sans index et sans en-tête pour rester "brut")
noms_extraits.to_csv('glaff_age_ment_light.txt', sep='|', index=False, header=False)

print(f"Étape 1 terminée : {len(noms_extraits)} noms extraits et sauvegardés dans 'glaff_age_ment_light.txt'.")

---

# À vous de jouer :

## Étape 1 - Préparation

1. Rendez-vous sur [Google Colab](https://colab.research.google.com/)
2. **Importez** le notebook `NB_glaff_age_ment.ipynb`qui se trouve sur la page du cours (séance 5).
3. **Chargez** les données extraites du GLAFF :
   1. À gauche, cliquez sur l'icône "Dossier" &#128193;
   2. Faites glisser le fichier de sous-corpus `glaff_age_ment_light.txt` dans la zone de téléchargement

## Étape 2 - Création du sous-corpus de dérivés "concurrents"

1. Exécutez le code ci-dessous (attention aux noms de fichiers) après avoir regardé les différentes étapes du script
2. Observez les résultats et commentez-les

In [ ]:
import pandas as pd

# 1. Chargement du sous-corpus
df_travail = pd.read_csv('glaff_age_ment_light.txt', sep='|', header=None)
df_travail.columns = ['Graphie', 'Code_GRACE', 'Lemme']

# 2. Création d'une colonne 'Radical' pour identifier les bases communes
# On enlève 'age', 'ement' ou 'ment' à la fin du mot
df_travail['Radical'] = df_travail['Graphie'].str.replace('age$', '', regex=True)\
                                             .str.replace('ement$', '', regex=True)\
                                             .str.replace('ment$', '', regex=True)

# 3. On identifie les doublons de radicaux
counts = df_travail['Radical'].value_counts()
doublons_liste = counts[counts >= 2].index

# 4. On filtre pour ne garder que les lignes dont le radical est dans la liste des doublons
noms_doublons = df_travail[df_travail['Radical'].isin(doublons_liste)].copy()

# 5. Tri pour l'affichage (les paires apparaissent ensemble)
noms_doublons = noms_doublons.sort_values(by='Radical')

print(f"Étape 2 terminée : {len(noms_doublons)} mots restants (couples identifiés).")
noms_doublons.head(50)

## Étape 3 - Observation

Dans la liste ainsi créée, vous devriez repérer **du bruit** :

- Quels mots en -age ne sont pas des dérivés qui nous intéressent ?
- Comment l'expliquez-vous ?

## Étape 4: _antrophopher_ ? _œsopher_ ?

In [ ]:
# 1. On filtre les noms se terminant par 'phage' dans notre liste de doublons
phages = noms_doublons[noms_doublons['Graphie'].str.endswith('phage', na=False)]

# 2. Affichage du compte et des résultats
print(f"Nombre de noms en -phage identifiés comme 'bruit' : {len(phages)}")
phages.head(30)

In [ ]:
# --- ÉTAPE DE NETTOYAGE DES FAUX POSITIFS ---

# On utilise le signe tilde (~) qui signifie "NON" en logique Python
# On ne garde que les lignes qui ne se terminent PAS par 'phage'
noms_doublons_propres = noms_doublons[~noms_doublons['Graphie'].str.endswith('phage', na=False)].copy()

# On recalcule les doublons après suppression (au cas où un 'phage' était le seul partenaire d'un mot)
counts_clean = noms_doublons_propres['Radical'].value_counts()
doublons_propres_liste = counts_clean[counts_clean >= 2].index
noms_doublons_propres = noms_doublons_propres[noms_doublons_propres['Radical'].isin(doublons_propres_liste)]

print(f"Nettoyage terminé : {len(noms_doublons) - len(noms_doublons_propres)} mots en -phage supprimés.")
print(f"Nombre de mots valides restants : {len(noms_doublons_propres)}")

## Étape 5: création d'un fichier CSV avec 50 couples de doubles dérivés pour analyse

In [ ]:
# 1. On récupère la liste des 50 premiers radicaux uniques sur le tableau NETTOYÉ
top_50_radicaux = noms_doublons_propres['Radical'].unique()[:50]

# 2. On filtre le tableau propre pour ne garder que les lignes de ces 50 radicaux
selection_50_couples = noms_doublons_propres[noms_doublons_propres['Radical'].isin(top_50_radicaux)]

# 3. Exportation vers le fichier CSV final
selection_50_couples.to_csv('top_50_doublons_age_ment_final.csv', 
                            index=False, 
                            sep=';', 
                            encoding='utf-8-sig')

print(f"✅ Terminé ! Le fichier 'top_50_doublons_age_ment_final.csv' contient {len(selection_50_couples)} lignes.")
print("Ce fichier ne contient plus les mots en -phage et se limite aux 50 premiers couples.")
selection_50_couples.head(10)

## Étape 6
1. Choisissez quatre couples de dérivés d'une même base
2. Annotez le fichier CSV en copiant-collant leur définition du Wiktionnaire
3. Commentez la différence de sens éventuelle en vous aidant du Wiktionnaire (ou d'un autre dictionnaire)